## นำเข้าไลบรารีที่จำเป็น  
นำเข้าไลบรารี PyTorch และ transformers ที่จำเป็นสำหรับการโหลดและใช้งานโมเดล Phi-4


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## ตั้งค่าเมล็ดสุ่ม
ตั้งค่าเมล็ดสุ่มเพื่อให้ได้ผลลัพธ์ที่สามารถทำซ้ำได้ในแต่ละครั้งที่รัน


In [ ]:
torch.random.manual_seed(0)

## โหลดโมเดลและตัวแปลงคำ Phi-4-mini-flash-reasoning  
โหลดโมเดล Microsoft Phi-4-mini-flash-reasoning และตัวแปลงคำที่เกี่ยวข้องจาก Hugging Face โมเดลจะถูกโหลดบน CUDA เพื่อการวิเคราะห์ที่รวดเร็วขึ้น


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## เตรียมข้อความสำหรับป้อนข้อมูล  
สร้างข้อความสนทนาที่มีโจทย์ปัญหาคณิตศาสตร์เกี่ยวกับสมการกำลังสอง และจัดรูปแบบโดยใช้เทมเพลตการสนทนาสำหรับโมเดล


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## สร้างการตอบกลับ  
สร้างการตอบกลับจากโมเดลโดยใช้พารามิเตอร์ที่กำหนด เช่น temperature และ top_p เพื่อควบคุมความสุ่มในผลลัพธ์


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## แปลงผลลัพธ์เป็นข้อความ
แปลงลำดับโทเค็นที่สร้างขึ้นกลับเป็นข้อความที่มนุษย์อ่านได้ โดยไม่รวมโทเค็นของข้อมูลต้นฉบับ เพื่อแสดงเฉพาะคำตอบของโมเดลเท่านั้น


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
